# Support Vector Machines: College Admission

Let's look at a classification example in Spark MLLib.  We looked at the college admission before. We can look again at this dataset.  


In [ ]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

## Step 1: Load the data

In [ ]:
dataset = spark.read.csv("/data/college-admissions/admission-data.csv", header=True, inferSchema=True)
dataset.printSchema()
dataset.show(20)

## Step 2 - Data Exploration

In [ ]:
# use describe
dataset.describe().toPandas()

In [ ]:
# see data spread
dataset.groupBy("admit").count().show()

## Step 3: Build the Vector

In [ ]:
from pyspark.ml.feature import VectorAssembler

## TODO : input cols : gre, gpa, rank

assembler = VectorAssembler(inputCols=['???', '???', '???'], outputCol="features")
featureVector = assembler.transform(dataset)
featureVector = featureVector.withColumn("label", featureVector["admit"])
featureVector.sample(False, 0.1, seed=10).show(50)

## Step 4: Split into training and test.

In [ ]:
## Split into training and test
## TODO: create training and test with an 80/20 split
(training, test) = featureVector.randomSplit([???, ???])

print ("training set count ", training.count())
print ("testing set count ", test.count())

## Step 5: Build the Linear SVM model

In [ ]:
from pyspark.ml.classification import LinearSVC

## TODO : set MaxIter to 10
lsvc = LinearSVC(maxIter=???, regParam=0.3)

In [ ]:
%%time

# Fit the model
## TODO : fit on 'training' set
print ("training starting...")
lsvcModel = lsvc.fit(???)
print("training done.")

In [ ]:
# Print the coefficients and intercept for linearsSVC
print ("inputs : gre, gpa, rank")
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

### Adjust Iterations

If any coefficient is zero, that variable won't be a factor in the decision !  
Set **maxIter=50**  and try again

## Step 6: Run the test set and get the predictions

In [ ]:
## TODO: transform the test data
## HINT : test
predictions_test = lsvcModel.transform(???)
predictions_test.show()

## sample
predictions_test.sampleBy("label", fractions={0: 0.5, 1: 0.5}, seed=0).show()


## Step 7: See the evaluation metrics

In [ ]:
predictions_test = lsvcModel.transform(???)  # Hint : test
predictions_train = lsvcModel.transform(???)  # Hint : training

### 7.1 - Accuracy

**TODO: Compare 'training' & 'test' set accuracies**  
Can you detect any overfitting / underfitting?


**TODO: Increase 'maxIterations' and try again**  
Does the accuracy go up?  

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

print("Training accuracy :",  evaluator.evaluate(predictions_train))
print("Test accuracy :",  evaluator.evaluate(predictions_test))

###  7.2 - Confusion matrix

In [ ]:
# Confusion matrix
cm = predictions_test.groupBy('label').pivot('prediction', [0,1]).count().na.fill(0).orderBy('label')
cm.show()

In [ ]:
import seaborn as sns

cm_pd = cm.toPandas()
cm_pd.set_index("label", inplace=True)
# print(cm_pd)

# colormaps : cmap="YlGnBu" , cmap="Greens", cmap="Blues",  cmap="Reds"
sns.heatmap(cm_pd, annot=True, cmap="Blues").plot()

### 7.3 - AUC
**=> What does AUC mean?** 

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# default metrics for BinaryClassificationEvaluator is 'areaUnderCurve'
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
# print ("default metrics : " ,evaluator.getMetricName())

print("AUC for training: " , evaluator.evaluate(predictions_train))
print ("AUC for test : " , evaluator.evaluate(predictions_test))

## Step 8: Try running a prediction on your own data

In [ ]:
import pandas as pd

newdata = pd.DataFrame({'gre' : [600, 700, 800], 
                        'gpa' : [4.0, 3.5, 3.2],
                        'rank': [1,   2,   3]}
             )
print(newdata)

## TODO : create a spark dataframe
## Hint : input is 'newdata'
spark_newdata = spark.createDataFrame(???)

## TODO : create feature vector
## Hint : spark_newdata
newfeatures = assembler.transform(???)

lsvcModel.transform(newfeatures).show()

## Step 9 : Understanding the impact of Scaling Data
Just now we have fed our input vector without scaling to SVM.  
IN this section we are going to scale the data and see if it improves the prediction.  
We will condense the code to focus on important stuff

### 9.1 : Raw data (without scaling)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC


print ("===== run with raw data (not scaled) =======")
dataset = spark.read.csv("/data/college-admissions/admission-data.csv", \
                         header=True, inferSchema=True)
assembler = VectorAssembler(inputCols=[ 'gre', 'gpa', 'rank'], outputCol="features")
featureVector = assembler.transform(dataset)
featureVector = featureVector.withColumn("label", featureVector["admit"])
(training, test) = featureVector.randomSplit([0.8, 0.2], seed=123)
lsvc = LinearSVC(maxIter=100, regParam=0.3, featuresCol='features')
lsvcModel = lsvc.fit(training)
print ("inputs :  gre, gpa, rank")
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

predictions_train = lsvcModel.transform(training)
predictions_test = lsvcModel.transform(test)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print ("Training AUC : " , evaluator.evaluate(predictions_train))
print ("Test AUC : " , evaluator.evaluate(predictions_test))  #AUC

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
print("Training accuracy",  evaluator.evaluate(predictions_train))
print("Testing accuracy",  evaluator.evaluate(predictions_test))
print ("===== END run with raw data (not scaled) =======")

### 9.2 : Scaled data

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import StandardScaler

print ("===== run with scaled data  =======")
dataset = spark.read.csv("/data/college-admissions/admission-data.csv", \
                         header=True, inferSchema=True)
assembler = VectorAssembler(inputCols=[ 'gre', 'gpa', 'rank'], outputCol="features")
featureVector = assembler.transform(dataset)
featureVector = featureVector.withColumn("label", featureVector["admit"])

# scaling
scaler = StandardScaler(inputCol="features", outputCol="featuresScaled",
                        withStd=True, withMean=False)
scalerModel = scaler.fit(featureVector)
fv_scaled = scalerModel.transform(featureVector)
fv_scaled.show(20, truncate=False)

(training, test) = fv_scaled.randomSplit([0.8, 0.2], seed=123)  ## CHANGED
lsvc = LinearSVC(maxIter=100, regParam=0.3, featuresCol='featuresScaled')  ## CHANGED

lsvcModel = lsvc.fit(training)
print ("inputs :  gre, gpa, rank")
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

predictions_train = lsvcModel.transform(training)
predictions_test = lsvcModel.transform(test)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print ("Training AUC : " , evaluator.evaluate(predictions_train))
print ("Test AUC : " , evaluator.evaluate(predictions_test))  #AUC


evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

print("Training accuracy",  evaluator.evaluate(predictions_train))
print("Testing accuracy",  evaluator.evaluate(predictions_test))

print ("===== END run with scaled data =======")


## Step 10 : Discuss the findings

### Coefficients
Here are the coefficients from one sample run

raw data run : 
inputs :  gre, gpa, rank
Coefficients: [0.00730924862823,0.803788881405,-0.182571791707]
Intercept: -7.016411699283878

scaled data run:
inputs :  gre, gpa, rank
Coefficients: [0.985025239033,0.311565356517,-0.180265498388]
Intercept: -7.500693768967119

### Accuracy
Compare accuracies